# Visualizing a single CT scan

I am very intrigued to visualize 1 CT scan in 3D. So far, we have visualized only cross-segments and projections. Let's spend some time to look up resources that will help us achieve visualizing in 3D.

- I came across a fantastic blog that will serve as the foundation for our analysis: [Vincente Rodriguez blog](https://vincentblog.xyz/posts/medical-images-in-python-computed-tomography).
- The [Wikipedia page](https://en.wikipedia.org/wiki/Marching_cubes) about `Marching cubes` is also a great introduction for 3D visualization.


## Importing the libraries